# Compute the correlation of the provided annotations and various IQA metrics.

In [5]:
import os
import numpy as np
import pandas as pd
from scipy.io import savemat, loadmat
from scipy.stats import zscore

#list of all IQA metrics to compute the correlation for
alglist=["SSIM_matlab","SSIM_python_pytorch","IW-SSIM_matlab","PSNR_matlab","PSNR_python_pytorch","MS-SSIM_matlab","MS-SSIM_python_pytorch","DISTS_matlab","DISTS_python_pytorch","DSS_matlab","FSIM_matlab","GMSD_matlab","HaarPSI_matlab","HaarPsi_python_tensorflow","LPIPS_python","MDSI_matlab","VIF_python_pytorch","VSI_matlab","niqe_matlab","PAQ2PIQ_python","BRISQUE_matlab"]

In [2]:
def compute_corr(df_annotations, method="spearman"):
    corr = df_annotations.corr(method=method)
    mask = np.tril(np.ones(corr.shape) * np.nan, k=0)
    mask[mask == 0] = 1
    return corr * mask

In [6]:
dataset_name_list=["live"]
corr_matrices_spear={}
corr_matrices_kendall={}
df_all_dict={}
for dataset_name in dataset_name_list:
        directory = os.fsencode("../annotations/"+dataset_name)
        dists = dict()
        df_name=pd.read_csv(directory.decode("utf-8")+"/"+os.listdir(directory)[0].decode("utf-8")).filename
        df_all =  pd.DataFrame(index = df_name)
        for file in os.listdir(directory):
                df_temp =pd.read_csv(directory.decode("utf-8")+"/"+file.decode("utf-8"))
                df_temp.index=df_temp.filename
                df_temp=df_temp.overall_quality
                df_temp.name="overall_quality_"+file.decode("utf-8")
                df_all=pd.concat([df_all,df_temp], axis=1)
        df_all=df_all.apply(zscore)
        df_all["mean_human"] = df_all.mean(axis=1)
        for alg in alglist:
                try:
                        directory = os.fsencode("../results/"+dataset_name+"/"+alg+"/")
                        dists = dict()
                        for file in os.listdir(directory):
                                namestr=loadmat(directory+file)["name"][0]
                                dists[namestr] = loadmat(directory+file)["algorithmvalue"][0,0]
                        df = pd.DataFrame(list(dists.items()), columns=['deg_name', alg])
                        df.set_index('deg_name', inplace=True)
                        df = df.sort_index()
                        df_all = pd.concat([df_all, df], axis=1)
                except:
                        print("results for "+alg+ " do not exist")
        df_all.replace([np.inf], 1000, inplace=True)
        df_all_dict[dataset_name]=df_all.copy()
        corr_matrix_spear = compute_corr(df_all,"spearman")
        corr_matrix_spear=corr_matrix_spear.loc[["mean_human"],alglist].transpose().rename({"mean_human":dataset_name},axis="columns")
        corr_matrices_spear[dataset_name]=corr_matrix_spear
        corr_matrix_kendall = compute_corr(df_all,"kendall")
        corr_matrix_kendall=corr_matrix_kendall.loc[["mean_human"],alglist].transpose().rename({"mean_human":dataset_name},axis="columns")
        corr_matrices_kendall[dataset_name]=corr_matrix_kendall
        display(dataset_name)
        display("spearman")
        table2_spear=corr_matrices_spear[dataset_name].round(2)
        display(table2_spear)
        display("kendall")
        table2_kendall=corr_matrices_kendall[dataset_name].round(2)
        display(table2_kendall)

'live'

'spearman'

,live
SSIM_python_pytorch,0.88


'kendall'

,live
SSIM_python_pytorch,0.71
